In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4

max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

dropout = 0.2

cpu


In [2]:
# read in our corpus into text string

with open('data\philosophers.txt','r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])



=== PLATO ===

How you, O Athenians, have been affected by my accusers, I cannot tell;
but I know that they almost made me forget who I was—so persuasively
did they speak; and yet they have hardly 


In [3]:
# list of unique characters in the corpus

chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', '—', '’', '“', '”', '\ufeff']


In [4]:
# encoder and decoder for strings to integers and vice versa

string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [5]:
# example of encoder and decoder at work

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(encoded_hello)
print(decoded_hello)

[61, 58, 65, 65, 68]
hello


In [6]:
# encode the entire text into a tensor

data = torch.tensor(encode(text), dtype=torch.long)

In [7]:
print(data[:100])

tensor([85,  0,  0, 23, 23, 23,  1, 40, 36, 25, 44, 39,  1, 23, 23, 23,  0,  0,
        32, 68, 76,  1, 78, 68, 74,  8,  1, 39,  1, 25, 73, 61, 58, 67, 62, 54,
        67, 72,  8,  1, 61, 54, 75, 58,  1, 55, 58, 58, 67,  1, 54, 59, 59, 58,
        56, 73, 58, 57,  1, 55, 78,  1, 66, 78,  1, 54, 56, 56, 74, 72, 58, 71,
        72,  8,  1, 33,  1, 56, 54, 67, 67, 68, 73,  1, 73, 58, 65, 65, 22,  0,
        55, 74, 73,  1, 33,  1, 64, 67, 68, 76])


In [8]:
# use first 80% of corpus for training; remaining 20% for validation
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

In [9]:
x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[72, 68, 65, 74, 73, 58,  1, 60],
        [54, 67,  1, 59, 54, 56, 74, 65],
        [54, 56, 73, 62, 75, 62, 73, 78],
        [66, 62, 77, 73, 74, 71, 58,  8]])
targets:
tensor([[68, 65, 74, 73, 58,  1, 60, 68],
        [67,  1, 59, 54, 56, 74, 65, 73],
        [56, 73, 62, 75, 62, 73, 78,  1],
        [62, 77, 73, 74, 71, 58,  8,  1]])


In [10]:
# sample of context and targets at work. context representing what the model sees, and target represents what the model is trying to predict.
x = train_data[:block_size]
y = train_data [1:block_size+1]

for t in range (block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context}, target is {target}')

when input is tensor([85]), target is 0
when input is tensor([85,  0]), target is 0
when input is tensor([85,  0,  0]), target is 23
when input is tensor([85,  0,  0, 23]), target is 23
when input is tensor([85,  0,  0, 23, 23]), target is 23
when input is tensor([85,  0,  0, 23, 23, 23]), target is 1
when input is tensor([85,  0,  0, 23, 23, 23,  1]), target is 40
when input is tensor([85,  0,  0, 23, 23, 23,  1, 40]), target is 36


In [11]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embeddings_table = nn.Embedding(vocabulary_size, vocabulary_size)

    def forward(self, index, targets=None):
        logits = self.token_embeddings_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B,T) array of indicdes in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

In [12]:
model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


v.-L“L-'Llæ)oL:lVit7dbnq'_I2d7?“yjIAfY:Z?U'(rpe)bN
gVZjpQaA.gHP*M0—G-1drM;9fk;N7-IS9iCXI8Qrfu,Rdbq l?dY5E]﻿Y“G﻿Q5﻿F3CU'L2er)TulBt”Du”j)Ad95s)oN8XlD:fnZfkh6=?jI﻿QwN1)DSilOx7?3Y(t*4e:Wqm2J'55)fUHrl5s yICU:WMfRCo)W;M:A,TFRM"Pz—TcAvæ!OE]q )Rq'F31k(’F-0xBMM79xwt?zCM"n"æyc.f2:tZwclUyHVKjMJ[)S=Ri)dh6ksKitTShbv2XgK-52-kwd9(cAwVi)Sm(V'P7:WWi L”IZ4MIs”fA'"X.HP9=e"E3[uK3?hn:pm8﻿v4H—SSv“57?
uF]V3'_rCEa_V*l3JFA3pch9rv.K*B0s.WS WqXezZi88jmKCq—kHIgEMJ]WQtL,Q3C-GEQmT-Y.HZjQrgTaOZL“h6u[As“;srxXwoU'—11'PlDS Rr=c”


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [21]:
# create the optimizer

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss {losses['train']:.4f}, validation loss {losses['val']:.4f}")
        

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # clear existing gradients in the optimizer - set to none to occupy less space
    loss.backward() #backward pass
    optimizer.step() # lets gradient descent work
print(loss.item())

step: 0, train loss 2.3941, validation loss 2.3943
step: 250, train loss 2.4280, validation loss 2.3898
step: 500, train loss 2.4226, validation loss 2.4170
step: 750, train loss 2.3899, validation loss 2.4025
step: 1000, train loss 2.4453, validation loss 2.4058
step: 1250, train loss 2.3836, validation loss 2.3982
step: 1500, train loss 2.4284, validation loss 2.3959
step: 1750, train loss 2.4261, validation loss 2.3951
step: 2000, train loss 2.4338, validation loss 2.3833
step: 2250, train loss 2.4230, validation loss 2.3849
step: 2500, train loss 2.3981, validation loss 2.3785
step: 2750, train loss 2.4091, validation loss 2.3945
step: 3000, train loss 2.4084, validation loss 2.4106
step: 3250, train loss 2.4040, validation loss 2.4023
step: 3500, train loss 2.4091, validation loss 2.3996
step: 3750, train loss 2.3925, validation loss 2.4037
step: 4000, train loss 2.3998, validation loss 2.3767
step: 4250, train loss 2.3885, validation loss 2.4098
step: 4500, train loss 2.4166, val

In [28]:
vocab = ['a', 'b', 'c']
vocab_size = len(vocab)
stoi = {ch:i for i,ch in enumerate(vocab)}   # char -> index
itos = {i:ch for i,ch in enumerate(vocab)}   # index -> char

def decode(indices):
    return ''.join(itos[i] for i in indices)



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embeddings_table = nn.Embedding(vocabulary_size, vocabulary_size)

    def forward(self, index, targets=None):
        print(f'index: {index}')
        logits = self.token_embeddings_table(index)  # (B,T,C)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        if targets is None:
            loss = None
        else:
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits.view(B, T, C), loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(index)          # (B,T,C)
            #print(f'index {index}')
            print(logits)
            logits = logits[:, -1, :]                # take last time step -> (B,C)
            #print('last')
            #print(logits)
            probs = F.softmax(logits, dim=-1)        # turn into probabilities
            #print('probs')
            #print(probs)
            index_next = torch.multinomial(probs, 1) # sample next token -> (B,1)
            #print(index_next)
            index = torch.cat((index, index_next), dim=1)
        return index


In [ ]:
torch.manual_seed(42)  # for reproducibility
model = BigramLanguageModel(vocab_size)

context = torch.zeros((1,1), dtype=torch.long)  # start with 'a' (index=0)
print(context)
output = model.generate(context, max_new_tokens=5)
print("Indices:", output.tolist())
print("Decoded:", decode(output[0].tolist()))


tensor([[0]])
index: tensor([[0]])
index tensor([[0]])
tensor([[[0.3367, 0.1288, 0.2345]]], grad_fn=<ViewBackward0>)
index: tensor([[0, 0]])
index tensor([[0, 0]])
tensor([[[0.3367, 0.1288, 0.2345],
         [0.3367, 0.1288, 0.2345]]], grad_fn=<ViewBackward0>)
index: tensor([[0, 0, 1]])
index tensor([[0, 0, 1]])
tensor([[[ 0.3367,  0.1288,  0.2345],
         [ 0.3367,  0.1288,  0.2345],
         [ 0.2303, -1.1229, -0.1863]]], grad_fn=<ViewBackward0>)
index: tensor([[0, 0, 1, 0]])
index tensor([[0, 0, 1, 0]])
tensor([[[ 0.3367,  0.1288,  0.2345],
         [ 0.3367,  0.1288,  0.2345],
         [ 0.2303, -1.1229, -0.1863],
         [ 0.3367,  0.1288,  0.2345]]], grad_fn=<ViewBackward0>)
index: tensor([[0, 0, 1, 0, 1]])
index tensor([[0, 0, 1, 0, 1]])
tensor([[[ 0.3367,  0.1288,  0.2345],
         [ 0.3367,  0.1288,  0.2345],
         [ 0.2303, -1.1229, -0.1863],
         [ 0.3367,  0.1288,  0.2345],
         [ 0.2303, -1.1229, -0.1863]]], grad_fn=<ViewBackward0>)
Indices: [[0, 0, 1, 0, 1,